# Pearson correlation

In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
sns.set_context('talk',font_scale=.8)
import sys, os, os.path
from scipy.io import wavfile
from scipy import signal
from mido import MidiFile

chops = []
bps = np.array([])

mid = MidiFile('200415a.mid')
print(mid.ticks_per_beat)
#ticks_per_second=mid.ticks_per_beat*2
ticks_per_second=210
time = 0
beat = 0

for i, track in enumerate(mid.tracks):
    print('Track {}: {}'.format(i, track.name))
    for msg in track:
        print(msg)
        if msg.type == 'note_off':
            beat += (msg.time / ticks_per_second)
            time += (msg.time / ticks_per_second)
            print ("time: " + str(msg.time))
          #  print (beat)
           # print (time)
        elif msg.type == 'note_on':
            beat += (msg.time / ticks_per_second)
            time += (msg.time / ticks_per_second)
            chops.append(time)
            bps = np.append(bps, 1/beat)
            beat = 0

print(time)
input_filename = '200415a_test.wav'
samrate, data = wavfile.read(str(input_filename))
print (len(data)/samrate)
print(np.sum(1/bps))
print(bps.size)

print("chops " + str(chops))


# bps = 1/(bps*(np.sum(1/bps)/(len(data)/samrate)))
# print(np.sum(1/bps))

tidal = '<'
for i in bps:
    tidal += str(i)
    tidal += ' '
tidal += '>'
print(tidal)

previous = 0
count = 1
input_filename = input_filename[:-4]
print("data length: " + str(data.size/samrate) + " " + str(data[0].size))

for i in chops:
    i *= samrate
    chop_index = range(int(previous), int(i))
    chop = [data[index] for index in chop_index]
    chop = np.array(chop)
    output_filename = input_filename + str(count) + '.wav' if count > 9 else input_filename + '0' + str(count) + '.wav'
    print(output_filename + ' length: ' + str(chop.size/samrate))
    wavfile.write(output_filename, samrate, chop)
    count += 1
    previous = i

i = len(data) - 1
chop_index = range(int(previous), int(i))
chop = [data[index] for index in chop_index]
chop = np.array(chop)
output_filename = input_filename + str(count) + '.wav'
wavfile.write(output_filename, samrate, chop)





96
Track 0: 2-Bells1 Attack 1 
<meta message track_name name='2-Bells1 Attack 1\x00' time=0>
<meta message time_signature numerator=4 denominator=4 clocks_per_click=36 notated_32nd_notes_per_beat=8 time=0>
<meta message time_signature numerator=4 denominator=4 clocks_per_click=36 notated_32nd_notes_per_beat=8 time=0>
note_on channel=0 note=62 velocity=60 time=445
note_off channel=0 note=62 velocity=64 time=10
time: 10
note_on channel=0 note=62 velocity=60 time=96
note_off channel=0 note=62 velocity=64 time=10
time: 10
note_on channel=0 note=62 velocity=60 time=110
note_off channel=0 note=62 velocity=64 time=10
time: 10
note_on channel=0 note=62 velocity=60 time=110
note_off channel=0 note=62 velocity=64 time=9
time: 9
note_on channel=0 note=62 velocity=60 time=114
note_off channel=0 note=62 velocity=64 time=9
time: 9
note_on channel=0 note=62 velocity=60 time=110
note_off channel=0 note=62 velocity=64 time=9
time: 9
note_on channel=0 note=62 velocity=60 time=104
note_off channel=0 note

200415a_test08.wav length: 1.1523809523809523
200415a_test09.wav length: 1.0952380952380953
200415a_test10.wav length: 1.1142857142857143
200415a_test11.wav length: 1.0952380952380953
200415a_test12.wav length: 1.1047619047619048
200415a_test13.wav length: 1.2
200415a_test14.wav length: 1.180952380952381
200415a_test15.wav length: 1.1238095238095238
200415a_test16.wav length: 1.0952380952380953
200415a_test17.wav length: 1.1428571428571428
200415a_test18.wav length: 1.1333333333333333
200415a_test19.wav length: 1.1333333333333333
200415a_test20.wav length: 1.1047619047619048
200415a_test21.wav length: 1.1428571428571428
200415a_test22.wav length: 1.1047619047619048
200415a_test23.wav length: 1.0571428571428572
200415a_test24.wav length: 2.3142857142857145
200415a_test25.wav length: 1.1047619047619048
200415a_test26.wav length: 1.1142857142857143
200415a_test27.wav length: 1.1428571428571428
200415a_test28.wav length: 1.180952380952381
200415a_test29.wav length: 1.1142857142857143
20041